# Character Extraction Agent 테스트 (Production Level)

캐릭터 추출 에이전트 (Level 1) - **고도화 버전** 테스트 노트북

## 주요 개선 사항
1. **Visual/Personality 분리** - 이미지 생성 AI용
2. **명시적 Relationships** - Neo4j 그래프 DB용
3. **Current Mood 추출** - TTS/표정 생성용
4. **Trait 보존 전략** - Re-extraction시 데이터 손실 방지

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

Project root: c:\jungle\weapon\sto-link-AI-backend


In [ ]:
SAMPLE_STORY = """
어두운 숲 속에서 서진은 검을 쥐고 있었다. 키가 크고 검은 머리카락을 가진 그는 어렸을 때부터 검술을 익혀온 뛰어난 검객이었다.
"우리가 이곳에서 그를 만날 수 있을까?" 서진의 옆에 서 있던 하나가 물었다.
하나는 마을에서 가장 현명한 치료사였고, 항상 차분하고 신중했다. 그녀는 하얀 치료사 로브를 입고 있었다.

갑자기 나무 사이에서 그림자가 나타났다. 그것은 이민호였다. 한때는 서진의 가장 친한 친구였지만,
지금은 그들의 마을을 배신한 적이 되어버렸다. 그의 차가운 눈빛에서 분노와 슬픔이 느껴졌다.

"오랜만이군, 서진." 이민호가 차갑게 말했다. 검은 갑옷을 입은 그는 예전보다 더 날카로워 보였다.
서진은 용감하게 앞으로 나섰다. 긴장감이 최고조에 달했다. "왜 마을을 배신했어, 민호?"

박서연이라는 젊은 여성이 숲에서 뛰어나왔다. 그녀는 마을의 정탐꾼으로, 빠르고 민첩했다.
"경비병들이 오고 있어요! 빨리 결정해야 해요!" 그녀의 목소리에는 급박함이 묻어났다.
"""

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/api/callback",
            "extracted_characters": [], "consistency_report": {}, "retry_count": 0, 
            "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

def pretty_print(title, data):
    print(f"\n{'='*70}")
    print(f"📋 {title}")
    print(f"{'='*70}")
    print(json.dumps(data, ensure_ascii=False, indent=2))

## 1. Production Level 캐릭터 추출 테스트

In [ ]:
from app.agents.extraction.character import character_extraction_node

async def test_character_extraction():
    print("🧑 Character Extraction (Production Level) 테스트...")
    state = create_base_state()
    return await character_extraction_node(state)

result = run_async(test_character_extraction())
pretty_print("Production Level 추출 결과", result)

🧑 Character Extraction (Production Level) 테스트...

📋 Production Level 추출 결과
{
  "extracted_characters": [
    {
      "name": "서진",
      "role": "protagonist",
      "status": "alive",
      "visual": {
        "appearance": [
          "tall",
          "dark hair"
        ],
        "attire": [
          "holding sword"
        ],
        "age_group": "adult",
        "gender": "male"
      },
      "personality": {
        "core_traits": [
          "skilled swordsman",
          "brave"
        ],
        "flaws": [
          "impulsive"
        ],
        "values": [
          "justice"
        ]
      },
      "relationships": [
        {
          "target": "이민호",
          "type": "ENEMY",
          "history": "former_friend",
          "strength": 8
        }
      ],
      "current_mood": {
        "emotion": "tense",
        "intensity": 7,
        "trigger": "confronting former friend"
      }
    },
    {
      "name": "하나",
      "role": "supporting",
      "status": "ali

## 2. 결과 분석 - 구조화된 데이터 확인

In [ ]:
characters = result.get("extracted_characters", [])

print("\n" + "="*70)
print("📊 Production Level 데이터 분석")
print("="*70)

for char in characters:
    print(f"\n🧑 {char.get('name')} ({char.get('role')})")
    print(f"   Status: {char.get('status')}")
    
    # Visual Traits (이미지 생성용)
    visual = char.get('visual', {})
    if visual:
        print(f"\n   🎨 Visual Traits (이미지 생성 AI용):")
        print(f"      - Appearance: {visual.get('appearance', [])}")
        print(f"      - Attire: {visual.get('attire', [])}")
        print(f"      - Age: {visual.get('age_group')}, Gender: {visual.get('gender')}")
    
    # Personality Traits
    personality = char.get('personality', {})
    if personality:
        print(f"\n   💭 Personality Traits (캐릭터 이해용):")
        print(f"      - Core: {personality.get('core_traits', [])}")
        print(f"      - Flaws: {personality.get('flaws', [])}")
        print(f"      - Values: {personality.get('values', [])}")
    
    # Relationships (Neo4j용)
    relationships = char.get('relationships', [])
    if relationships:
        print(f"\n   🔗 Relationships (Neo4j 그래프용):")
        for rel in relationships:
            history = f" (was: {rel.get('history')})" if rel.get('history') else ""
            print(f"      → {rel.get('target')}: {rel.get('type')} (strength: {rel.get('strength', 'N/A')}){history}")
    
    # Current Mood
    mood = char.get('current_mood', {})
    if mood:
        print(f"\n   😊 Current Mood (TTS/표정용):")
        print(f"      - Emotion: {mood.get('emotion')} (intensity: {mood.get('intensity')}/10)")
        print(f"      - Trigger: {mood.get('trigger')}")


📊 Production Level 데이터 분석

🧑 서진 (protagonist)
   Status: alive

   🎨 Visual Traits (이미지 생성 AI용):
      - Appearance: ['tall', 'dark hair']
      - Attire: ['holding sword']
      - Age: adult, Gender: male

   💭 Personality Traits (캐릭터 이해용):
      - Core: ['skilled swordsman', 'brave']
      - Flaws: ['impulsive']
      - Values: ['justice']

   🔗 Relationships (Neo4j 그래프용):
      → 이민호: ENEMY (strength: 8) (was: former_friend)

   😊 Current Mood (TTS/표정용):
      - Emotion: tense (intensity: 7/10)
      - Trigger: confronting former friend

🧑 하나 (supporting)
   Status: alive

   🎨 Visual Traits (이미지 생성 AI용):
      - Appearance: []
      - Attire: ["wearing white healer's robe"]
      - Age: adult, Gender: female

   💭 Personality Traits (캐릭터 이해용):
      - Core: ['wise', 'cautious']
      - Flaws: []
      - Values: ['healing']

   😊 Current Mood (TTS/표정용):
      - Emotion: concerned (intensity: 5/10)
      - Trigger: impending confrontation

🧑 이민호 (antagonist)
   Status: alive

   🎨 V

## 3. Neo4j 그래프 쿼리 시뮬레이션

In [ ]:
print("\n" + "="*70)
print("🔍 Neo4j 쿼리 시뮬레이션")
print("="*70)

# 모든 관계 수집
all_relationships = []
for char in characters:
    for rel in char.get('relationships', []):
        all_relationships.append({
            'source': char.get('name'),
            'target': rel.get('target'),
            'type': rel.get('type'),
            'history': rel.get('history'),
            'strength': rel.get('strength')
        })

print("\n📌 '서진의 적들은 누구인가?' 쿼리:")
enemies = [r for r in all_relationships if r['source'] == '서진' and r['type'] in ['ENEMY', 'BETRAYER', 'RIVAL']]
for e in enemies:
    print(f"   → {e['target']} ({e['type']})")
if not enemies:
    # 역방향 체크
    enemies_of = [r for r in all_relationships if r['target'] == '서진' and r['type'] in ['ENEMY', 'BETRAYER']]
    for e in enemies_of:
        print(f"   → {e['source']} ({e['type']})")

print("\n📌 '과거에 친구였던 관계' 쿼리:")
former_friends = [r for r in all_relationships if r.get('history') and 'friend' in r['history'].lower()]
for f in former_friends:
    print(f"   → {f['source']} ↔ {f['target']}: {f['type']} (was: {f['history']})")

print("\n📌 Cypher 쿼리 예시:")
print("""   
   // 서진의 모든 적 찾기
   MATCH (c:Character {name: '서진'})-[:ENEMY|BETRAYER]->(enemy)
   RETURN enemy.name
   
   // 과거 친구였던 현재 적 관계
   MATCH (a:Character)-[r:ENEMY {history: 'former_friend'}]->(b)
   RETURN a.name, b.name
""")


🔍 Neo4j 쿼리 시뮬레이션

📌 '서진의 적들은 누구인가?' 쿼리:
   → 이민호 (ENEMY)

📌 '과거에 친구였던 관계' 쿼리:
   → 서진 ↔ 이민호: ENEMY (was: former_friend)
   → 이민호 ↔ 서진: ENEMY (was: former_friend)

📌 Cypher 쿼리 예시:
   
   // 서진의 모든 적 찾기
   MATCH (c:Character {name: '서진'})-[:ENEMY|BETRAYER]->(enemy)
   RETURN enemy.name

   // 과거 친구였던 현재 적 관계
   MATCH (a:Character)-[r:ENEMY {history: 'former_friend'}]->(b)
   RETURN a.name, b.name



## 4. Re-extraction 테스트 (Trait 보존)

In [ ]:
async def test_re_extraction():
    """Re-extraction with production-level data preservation"""
    print("🔄 Re-extraction (Production Level) 테스트...")
    
    state = create_base_state()
    state["retry_count"] = 1
    
    # 이전 추출 결과 (Production Level 포맷)
    state["extracted_characters"] = [
        {
            "name": "이민호", 
            "role": "antagonist",
            "status": "alive",
            "visual": {
                "appearance": ["cold eyes", "sharp features"],
                "attire": ["black armor"],
                "age_group": "adult",
                "gender": "male"
            },
            "personality": {
                "core_traits": ["cold", "skilled"],
                "flaws": ["bitter", "vengeful"],
                "values": ["revenge"]
            },
            "relationships": [
                {"target": "서진", "type": "FRIEND", "strength": 8}  # 잘못됨 - ENEMY여야 함
            ],
            "current_mood": {
                "emotion": "hostile",
                "intensity": 8,
                "trigger": "confronting former friend"
            }
        }
    ]
    
    # 충돌 정보
    state["consistency_report"] = {
        "conflicts": [
            {
                "type": "RELATIONSHIP_CONFLICT",
                "description": "이민호와 서진의 관계가 FRIEND로 되어있으나, 스토리상 배신 후 ENEMY가 되었음. history에 'former_friend' 추가 필요.",
                "severity": "HIGH",
                "affected_elements": ["이민호", "서진"]
            }
        ]
    }
    
    return await character_extraction_node(state)

re_result = run_async(test_re_extraction())
pretty_print("Re-extraction 결과", re_result)

🔄 Re-extraction (Production Level) 테스트...
[CHARACTER] Re-extracting with 1 conflicts as feedback

📋 Re-extraction 결과
{
  "extracted_characters": [
    {
      "name": "이민호",
      "role": "antagonist",
      "status": "alive",
      "visual": {
        "appearance": [
          "cold eyes",
          "sharp features"
        ],
        "attire": [
          "black armor"
        ],
        "age_group": "adult",
        "gender": "male"
      },
      "personality": {
        "core_traits": [
          "cold",
          "skilled",
          "betrayer"
        ],
        "flaws": [
          "bitter",
          "vengeful"
        ],
        "values": [
          "revenge"
        ]
      },
      "relationships": [
        {
          "target": "서진",
          "type": "ENEMY",
          "history": "former_friend",
          "strength": 8
        }
      ],
      "current_mood": {
        "emotion": "hostile",
        "intensity": 8,
        "trigger": "confronting former friend"
      },

In [ ]:
# 데이터 보존 분석
print("\n" + "="*70)
print("📊 데이터 보존 분석")
print("="*70)

re_characters = re_result.get("extracted_characters", [])

for char in re_characters:
    if "민호" in char.get("name", ""):
        print(f"\n🧑 {char.get('name')}:")
        
        # Visual 보존 확인
        visual = char.get('visual', {})
        print(f"   🎨 Visual (보존되어야 함):")
        print(f"      - Appearance: {visual.get('appearance', [])}")
        print(f"      - Attire: {visual.get('attire', [])}")
        
        # Personality 보존 확인
        personality = char.get('personality', {})
        print(f"   💭 Personality (보존되어야 함):")
        print(f"      - Core: {personality.get('core_traits', [])}")
        print(f"      - Flaws: {personality.get('flaws', [])}")
        
        # Relationship 수정 확인
        relationships = char.get('relationships', [])
        print(f"   🔗 Relationships (수정되어야 함):")
        for rel in relationships:
            if rel.get('target') == '서진':
                expected_type = "ENEMY"
                actual_type = rel.get('type')
                history = rel.get('history')
                status = "✅" if actual_type in ['ENEMY', 'BETRAYER'] else "❌"
                print(f"      {status} {rel.get('target')}: {actual_type} (history: {history})")
        
        # 변경 사항
        if char.get('trait_changes'):
            print(f"   📝 변경 사항: {char.get('trait_changes')}")


📊 데이터 보존 분석

🧑 이민호:
   🎨 Visual (보존되어야 함):
      - Appearance: ['cold eyes', 'sharp features']
      - Attire: ['black armor']
   💭 Personality (보존되어야 함):
      - Core: ['cold', 'skilled', 'betrayer']
      - Flaws: ['bitter', 'vengeful']
   🔗 Relationships (수정되어야 함):
      ✅ 서진: ENEMY (history: former_friend)
   📝 변경 사항: Changed 'FRIEND' relationship to 'ENEMY' with 'former_friend' history. Preserved all other valid traits.
